## Hangman

In [ ]:
# Setup imports and path
import sys, os
from pathlib import Path

PROJECT_ROOT = Path('/home/mila/b/baldelld/scratch/hangman')
SRC_DIR = PROJECT_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

from hangman.sct.utils import (
    extract_all_secrets_from_text,
    extract_last_secret,
    summarize_secret_history,
    infer_pattern_from_text,
    estimate_candidates_from_transcript,
)

print('Loaded utils from:', SRC_DIR)

Loaded utils from: /home/mila/b/baldelld/scratch/hangman/src


In [2]:
# Secret extraction basic tests
examples = [
    """
<secret>apple</secret>
""",
    """
 Thoughts...
 <secret>apple</secret>
 ...later...
 <secret>apples</secret>
""",
    """
No secret here
""",
]

for i, text in enumerate(examples):
    all_s = extract_all_secrets_from_text(text)
    last_s = extract_last_secret(text)
    print(f"Example {i}: all={all_s}, last={last_s}")

history = [
    None,
    "<secret>apple</secret>",
    "<secret>apple</secret>",
    "<secret>apples</secret>",
    "<secret>apples</secret> <secret>apple</secret>",
]
print('\nSummary:', summarize_secret_history(history))


Example 0: all=['apple'], last=apple
Example 1: all=['apple', 'apples'], last=apples
Example 2: all=[], last=None

Summary: {'secrets_by_turn': [(1, 'apple'), (2, 'apple'), (3, 'apples'), (4, 'apple')], 'unique_secrets': ['apple', 'apples'], 'last_secret': 'apple', 'secret_defined': True, 'secret_stable': False, 'secret_changes_count': 2, 'first_secret_turn': 1, 'multi_tag_in_state': True}


In [3]:
# Pattern inference and candidate estimation (deterministic)
transcript = (
    "--- TURN 1 (Player) ---\n"
    "Let's play Hangman.\n\n"
    "--- TURN 2 (Agent) ---\n"
    "Okay, the pattern is: _ a _ e _.\n\n"
    "--- TURN 3 (Player) ---\n"
    "My next guess is the single letter \"t\".\n\n"
    "--- TURN 4 (Agent) ---\n"
    "Updated pattern: _ a t e _.\n\n"
)

pattern = infer_pattern_from_text(transcript)
print('Pattern:', pattern)

# If you have a dictionary path locally, set it; otherwise this will return []
DICT = None  # e.g., '/usr/share/dict/words'
cands = estimate_candidates_from_transcript(
    transcript_text=transcript,
    n=5,
    method='deterministic',
    dictionary_path=DICT,
)
print('Deterministic candidates (first 5):', cands)


Pattern: _ a t e _
Deterministic candidates (first 5): []


In [6]:
# Optional: load an LLM provider for fallbacks (configure in config/config.yaml)
from hangman.providers.llmprovider import load_llm_provider

PROVIDERS_CONFIG = str(PROJECT_ROOT / 'config' / 'config.yaml')
LLM_PROVIDER_NAME = 'qwen3_235b_openrouter_low'  # change if needed
LLM_PROVIDER_NAME = 'gpt_oss_120b_openrouter'  # change if needed

llm = None
try:
    llm = load_llm_provider(PROVIDERS_CONFIG, LLM_PROVIDER_NAME)
    print('Loaded LLM provider:', LLM_PROVIDER_NAME)
except Exception as e:
    print('Skipping LLM tests (provider not available):', e)


Loaded LLM provider: gpt_oss_120b_openrouter


In [7]:
# LLM-based candidate generation test (if provider is loaded)
from hangman.sct.utils import format_interaction_log

if llm is not None:
    log = [
        ["Let's play Hangman.", None],
        ["Pattern: _ a _ e _", "<secret>sample</secret>"],
        ["My next guess is the single letter \"t\".", None],
        ["Updated: _ a t e _", "<secret>sample</secret>"],
    ]
    transcript = format_interaction_log(log)
    llm_cands = estimate_candidates_from_transcript(
        transcript_text=transcript,
        n=5,
        method='llm',
        llm_provider=llm,
        llm_max_n=10,
    )
    print('LLM candidates (first 5):', llm_cands)
else:
    print('LLM provider not available; skipped LLM candidate test.')


--- From OpenRouter Client ---
Wire messages: [{'role': 'user', 'content': 'From the Hangman transcript below, infer the constraints and return up to 10 plausible secret words (lowercase, no spaces or hyphens) that are all consistent with the dialogue so far.\n\nConstraints to respect when proposing each word:\n- Word length must match the most recent pattern (count underscores and revealed letters).\n- Letters confirmed PRESENT must appear in the word in the exact positions indicated by the pattern.\n- Letters confirmed ABSENT must not appear anywhere in the word.\n- Avoid duplicates; return distinct words only.\n\nReturn ONLY a JSON array of strings, e.g., ["apple", "angle"]. No extra text.\n\n<transcript>\n--- TURN 1 (Player) ---\nPLAYER UTTERANCE:\nLet\'s play Hangman.\n\n--- TURN 2 (Agent) ---\nAGENT UTTERANCE:\nPattern: _ a _ e _\n\n--- TURN 3 (Player) ---\nPLAYER UTTERANCE:\nMy next guess is the single letter "t".\n\n--- TURN 4 (Agent) ---\nAGENT UTTERANCE:\nUpdated: _ a t e _\n

2025-09-22 13:00:55,683 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


LLM candidates (first 5): ['cater', 'later', 'hater', 'eater', 'water']


In [6]:
transcript

'--- TURN 1 (Player) ---\nPLAYER UTTERANCE:\nLet\'s play Hangman.\n\n--- TURN 2 (Agent) ---\nAGENT UTTERANCE:\nPattern: _ a _ e _\n\nAGENT\'S PRIVATE MEMORY:\n<secret>sample</secret>\n\n--- TURN 3 (Player) ---\nPLAYER UTTERANCE:\nMy next guess is the single letter "t".\n\n--- TURN 4 (Agent) ---\nAGENT UTTERANCE:\nUpdated: _ a t e _\n\nAGENT\'S PRIVATE MEMORY:\n<secret>sample</secret>\n'

In [8]:
# LLM-based secret extraction fallback test (if provider is loaded)
from hangman.sct.utils import summarize_secret_history_with_llm

if llm is not None:
    priv_states = [
        None,
        "Notes... <secret>alpha</secret>",
        "More... <secret>alpha</secret>",
        "Change... <secret>omega</secret>",
        "Ambiguous: <secret>omega</secret> <secret>alpha</secret>",
    ]
    summary_llm = summarize_secret_history_with_llm(priv_states, llm_provider=llm)
    print('LLM secret summary:', summary_llm)
else:
    print('LLM provider not available; skipped LLM secret extraction test.')


--- From OpenRouter Client ---
Wire messages: [{'role': 'user', 'content': "From the sequence of the assistant's private working memory snapshots below (chronological),\nextract all distinct secret words that appear. Return ONLY a JSON array of lowercase words,\nin order of first appearance. Do not include any other text.\n\n<private_states>\n[STATE 1]\nNotes... <secret>alpha</secret>\n\n[STATE 2]\nMore... <secret>alpha</secret>\n\n[STATE 3]\nChange... <secret>omega</secret>\n\n[STATE 4]\nAmbiguous: <secret>omega</secret> <secret>alpha</secret>\n</private_states>"}]
--- End OpenRouter Client ---


2025-09-22 13:03:13,214 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


-- LLM secret extraction succeeded --
LLM raw response: ["alpha","omega"] 
--
LLM secret summary: {'secrets_by_turn': [(0, 'alpha'), (1, 'omega')], 'unique_secrets': ['alpha', 'omega'], 'last_secret': 'omega', 'secret_defined': True, 'secret_stable': False, 'secret_changes_count': 1, 'first_secret_turn': 0, 'multi_tag_in_state': False}


### Letter Frequency

In [ ]:
LETTER_FREQUENCIES = {
    'E': 2.02, 
    'T': 9.10, 
    'A': 8.12, 
    'O': 7.68, 
    'I': 7.31, 
    'N': 6.95, 
    'S': 6.28, 
    'R': 6.02, 
    'H': 5.92, 
    'D': 4.32, 
    'L': 3.98, 
    'U': 2.88, 
    'C': 2.71, 
    'M': 2.61, 
    'F': 2.30, 
    'Y': 2.11, 
    'W': 2.09, 
    'G': 2.03, 
    'P': 1.82, 
    'B': 1.49, 
    'V': 1.11, 
    'K': 0.69, 
    'X': 0.17, 
    'Q': 0.11, 
    'J': 0.10, 
    'Z': 0.07, 
}


### Player

### Dictionary

In [3]:
from wordfreq import top_n_list
words = top_n_list("en", 300000)  # 50k most common words

In [ ]:
sub_words = [w for w in words if (len(w) == 5) ]
sub_words = [w for w in sub_words if (w[3] == 'a')]

## Diagnosis Simulator

### Diagnosis SCT quick checks


In [1]:
# Imports for diagnosis SCT tests
import sys
from pathlib import Path

PROJECT_ROOT = Path('/home/mila/b/baldelld/scratch/hangman')
SRC_DIR = PROJECT_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

from hangman.sct import diagnosis_utils as dxu
from hangman.prompts import diagnosis_simulator_sct as dxp
from hangman.players.deterministic_diagnosis_simulator_player import DeterministicDiagnosisSimulatorPlayer

print('Loaded diagnosis utils and player from:', SRC_DIR)


Loaded diagnosis utils and player from: /home/mila/b/baldelld/scratch/hangman/src


In [ ]:
# WM summarization with tag 'diagnosis'
priv_states = [
    None,
    "<diagnosis>influenza</diagnosis>",
    "notes... <diagnosis>influenza</diagnosis>",
    "update... <diagnosis>pulmonary embolism</diagnosis>",
]
print(dxu.summarize_secret_history(priv_states))


{'secrets_by_turn': [(1, 'influenza'), (2, 'influenza'), (3, 'pulmonary embolism')], 'unique_secrets': ['influenza', 'pulmonary embolism'], 'last_secret': 'pulmonary embolism', 'secret_defined': True, 'secret_stable': False, 'secret_changes_count': 1, 'first_secret_turn': 1, 'multi_tag_in_state': False}


In [3]:
# Reveal parsing (multi-word)
print(dxu.parse_revealed_secret('Pulmonary embolism'))
print(dxu.parse_revealed_secret('"Type-2 diabetes"'))
print(dxu.parse_revealed_secret('` RSV `'))


pulmonary embolism
type
rsv


In [4]:
# Deterministic candidate filtering using a tiny mock catalog
conditions_catalog = {
    'influenza': {'E_FEVER', 'E_COUGH', 'E_MYALGIA'},
    'pulmonary embolism': {'E_DYSPNEA', 'E_CHEST_PAIN'},
    'migraine': {'E_HEADACHE', 'E_PHOTOPHOBIA'},
}

used_features = {
    'E_FEVER': 'yes',
    'E_CHEST_PAIN': 'no',
}

cands = dxu.estimate_candidates_from_transcript(
    transcript_text='dummy',
    n=5,
    method='deterministic',
    used_features=used_features,
    conditions_catalog=conditions_catalog,
)
print('Filtered candidates:', cands)


Filtered candidates: ['influenza']


In [5]:
# Deterministic player: question selection and updates (using mock CSVs)
# We'll simulate dataset by writing small temporary CSVs in-memory environment is not available here,
# so this is a conceptual smoke test; adapt paths to your local ddxplus CSVs when running.
from tempfile import TemporaryDirectory
import csv

with TemporaryDirectory() as td:
    evidences_path = Path(td) / 'evidences.csv'
    conditions_path = Path(td) / 'conditions.csv'

    with open(evidences_path, 'w', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow(['code_question','question_en'])
        w.writerow(['E_FEVER','Do you have fever?'])
        w.writerow(['E_COUGH','Do you have cough?'])
        w.writerow(['E_MYALGIA','Do you have muscle pain?'])
        w.writerow(['E_DYSPNEA','Do you have shortness of breath?'])
        w.writerow(['E_CHEST_PAIN','Do you have chest pain?'])

    with open(conditions_path, 'w', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow(['condition_name','related'])
        w.writerow(['influenza',"['E_FEVER','E_COUGH','E_MYALGIA']"])
        w.writerow(['pulmonary embolism',"['E_DYSPNEA','E_CHEST_PAIN']"]) 

    player = DeterministicDiagnosisSimulatorPlayer(
        evidences_csv_path=str(evidences_path),
        conditions_csv_path=str(conditions_path),
        random_seed=42,
        t_fork=4,
    )

    # Turn 0: opening
    print(player.opening()[:60] + '...')

    # Ask a question
    q1 = player.next_guess()
    print('Q1:', q1)

    # Simulate agent says yes
    player.update_with_agent_answer('Yes.')

    # Ask another question
    q2 = player.next_guess()
    print('Q2:', q2)

    # Simulate agent says no
    player.update_with_agent_answer('No, not present.')

    # Final guess formatting
    print('Final prompt:', player.final_guess('influenza'))


You are the attending physician hosting a diagnostic session...
Q1: Do you have shortness of breath?
Q2: Do you have fever?
Final prompt: Is the medical condition "influenza"?
Answer only with a single word: yes or no.
